<a href="https://colab.research.google.com/github/android-kunjapppan/PyTorch-Tutorial/blob/master/PyTorch_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Netoworks:
* Mainly work on image data and applys a operation called Convolutions
      Initial Image * Filter = Resulting Image
      (W-F+2P)/S + 1

* Max Pooling is used to down sample an Image by appplying maximum filter to sub-regions. REduces the size of the image. And tries to avoid overfitting.



In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# Device Config

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters


In [3]:
num_epochs =4
batch_size = 4
learning_rate = 0.001

# Transforms:
* Dataset has PILImage images of range [0,1].
* We transform them to the Tensors of Normalized range [-1,1]

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Dataset:

In [6]:
train_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = True,
                                             download = True,
                                             transform = transform
                                             )

test_dataset = torchvision.datasets.CIFAR10(root = './data',
                                             train = False,
                                             download = True,
                                             transform = transform
                                             )

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# DataLoader

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True
                                          )

test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False
                                          )

In [8]:
# Dataset Classes
classes = ('plane','car','bird','cat','deer',
           'dog','frog','horse','ship','truck')

# ConvNet Model

In [18]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet,self).__init__()
    self.conv1 = nn.Conv2d(in_channels =3, out_channels= 6,kernel_size= 5,) # 3 because the image has 3 channels 'RGB'
    # shape after this operation [4,6,28,28]
    
    self.pool = nn.MaxPool2d(2,2)
    # [4,6,14,14]
    
    self.conv2 = nn.Conv2d(6, 16, 5)
    # [4,16,10,10]
    
    # will apply one more poolin layer in our network so the input size becomes [4,16,5,5]
    
    # 16*5*5 because the the o/p of previous layer will be [4,16,5,5] so will multiply 16*5*5
    self.fc1 = nn.Linear(16*5*5,120) 
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    out = self.pool(F.relu(self.conv1(x)))
    out = self.pool(F.relu(self.conv2(out)))
    
    # FLattening the tensor
    out = out.view(-1,16*5*5) 
    
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(x)

    return x

In [20]:
model = ConvNet()

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr = learning_rate)
n_total_steps = len(train_loader)

# Training:

In [21]:
for epoch in range(num_epochs):
  for i ,(images,labels) in enumerate(train_loader):

    # Forward Pass
    outputs = model(images)
    loss = criterion(outputs,labels)

    # Backward and Optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%2000 ==0:
      print(f'epochs : {epoch+1}/{num_epochs}, Step :{i+1}/{n_total_steps},Loss :{loss.item():.4f}')


RuntimeError: ignored